<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/4_LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [ ]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

Riferimento: https://stackoverflow.com/questions/30483246/how-to-check-if-a-python-module-has-been-imported

In [ ]:
catboost = 'catboost'
if catboost not in sys.modules:
  !pip install catboost 

## Dataset import

In [ ]:
from sklearn import preprocessing

data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

min_max_scaler = preprocessing.MinMaxScaler()
time = data['TransactionDT'].values.reshape(-1,1)
data['TransactionDT'] = min_max_scaler.fit_transform(time)
data.drop(columns=['TransactionID'], inplace=True)

## Model import

### Catboost

In [ ]:
load_catboost = False
if load_catboost:
  cat = CatBoostClassifier()
  cat.load_model('catboost')  

## LIME

**LIME Local interpretable model-agnostic explanations**

1. LIME generates a new dataset consisting of: 
  * permuted samples, drawing from a normal distribution with mean and standard deviation taken from the feature (in case of tabular data);
  * the corresponding predictions of the black box model;
2. On this new dataset LIME then trains an interpretable model;
3. The model is weighted by the proximity of the sampled instances to the instance of interest.

The **recipe for training local surrogate models**:
1. Select your instance of interest for which you want to have an explanation of its black box prediction.
2. Perturb your dataset and get the black box predictions for these new points.
3. Weight the new samples according to their proximity to the instance of interest.
4. Train a weighted, interpretable model on the dataset with the variations.
5. Explain the prediction by interpreting the local model.

Tutorial LIME: https://blog.dominodatalab.com/shap-lime-python-libraries-part-2-using-shap-lime/

Documentazione LIME: https://lime-ml.readthedocs.io/en/latest/lime.html

In [ ]:
def lime(X_train, X_test, categorical_features, clf, n_features):
  import lime
  import lime.lime_tabular

  # setup dell'esxplainer
  explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.values.tolist(),
                                                      class_names=['isFraud'],
                                                      categorical_features=categorical_features,
                                                      verbose=True, mode='classification',
                                                      discretize_continuous = True, discretizer='decile')
  
  # explanation
  exp = explainer.explain_instance(X_test.values[j], clf.predict_proba, num_features=n_features)
  exp.show_in_notebook(show_table=True)
  return  

In [ ]:
lime(X_train, X_test, cat_sign_col, cat, 10)